In [9]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as nnfunc
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim import SGD
from torch.optim import lr_scheduler
from torch.utils.data import random_split, Subset

In [10]:
classes = ['not_wearing','wearing_eyeglasses','wearing_hat']

transformer = transforms.Compose([
    transforms.ToTensor(),
#     transforms.Resize((32,32)),
    transforms.Normalize([0.5,0.5,0.5],
                        [0.5,0.5,0.5])
])

train_path = 'data_wearing/train'
test_path = 'data_wearing/test'

train_data = torchvision.datasets.ImageFolder(train_path, transform=transformer)

train_ratio = 0.8
train_size = int(train_ratio * len(train_data))
val_size = len(train_data) - train_size
train_set, val_set = random_split(train_data, [train_size, val_size])

trainloader = DataLoader(
    train_set,
    batch_size = 64,
    shuffle = True
)

valloader = DataLoader(
    val_set,
    batch_size = 64,
    shuffle = True
)

In [11]:
def ResBlock(in_channels, out_channels, pool=False):
    layers = []
    layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
    layers.append(nn.BatchNorm2d(out_channels))
    layers.append(nn.ReLU(inplace=True))
    if pool: 
        layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

class ResNet9(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.conv1 = ResBlock(in_channels, 64)
        self.conv2 = ResBlock(64, 128, pool=True)
        self.res1 = nn.Sequential(ResBlock(128, 128), ResBlock(128, 128))
        self.conv3 = ResBlock(128, 256, pool=True)
        self.conv4 = ResBlock(256, 512, pool=True)
        self.res2 = nn.Sequential(ResBlock(512, 512), ResBlock(512, 512))
        self.fc = nn.Sequential(nn.MaxPool2d(4), 
                                        nn.Flatten(), 
                                        nn.Dropout(0.2),
                                        nn.Linear(512, num_classes))
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.fc(out)
        return out

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# model = ResNet9(3, 3)

# ResNet
model = torchvision.models.resnet18(pretrained=False)
inchannel = model.fc.in_features
model.fc = nn.Linear(inchannel, 3)

# Continue training
# best_model = torch.load('model/task2.model')
# model.load_state_dict(best_model, strict=False)

model.to(device)
torch.save(model.state_dict(), 'model/task2-resnet18-adam-30.model')

lr = 0.01
weight_decay = 1e-4
num_epoches = 30
grad_clip = 0.1

train_count = len(train_set)
val_count = len(val_set)

optimizer = Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
# optimizer = Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = lr_scheduler.OneCycleLR(optimizer, lr, epochs=num_epoches, steps_per_epoch=train_count)

loss_function = nn.CrossEntropyLoss()

best_accuracy = 0.0

hist = []

# for epoch in range(30,45):
for epoch in range(num_epoches):
    model.to(device)
    model.train()
    train_accuracy = 0.0
    train_loss = 0.0
    for i, (images, labels) in enumerate(trainloader):
        if torch.cuda.is_available():
            images = images.to(device)
            labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # gradient clipping
        nn.utils.clip_grad_value_(model.parameters(), grad_clip)

        train_loss += loss.cpu().data * images.size(0)
        _, prediction = torch.max(outputs.data,1)
        train_accuracy += int(torch.sum(prediction == labels.data))
    
    train_accuracy = train_accuracy / train_count
    train_loss = train_loss/train_count
    
    model.eval()
    val_accuracy = 0.0
    with torch.no_grad():
        for i, (images, labels) in enumerate(valloader):
            if torch.cuda.is_available():
                images = images.to(device)
                labels = labels.to(device)
            outputs = model(images)
            _,prediction = torch.max(outputs.data,1)
            val_accuracy += int(torch.sum(prediction==labels.data))

        val_accuracy = val_accuracy / val_count
        
        print(f"Epoch {epoch}")
        print(f"Train_accuracy {train_accuracy}")
        print(f"Train Loss {train_loss}")
        print(f"Val Accuracy {val_accuracy}")
        
        hist.append({ 
            'epoch':epoch,
            'train_accuracy':train_accuracy,
            'val_accuracy':val_accuracy,
            'best_accuracy':best_accuracy,
            'train_loss':train_loss
        })
        
        if val_accuracy > best_accuracy:
            print(f"Best Val Accuracy {val_accuracy}")
            torch.save(model.state_dict(), 'model/task2-resnet18-adam-30.model')
            best_accuracy = val_accuracy

Epoch 0
Train_accuracy 0.6874003189792663
Train Loss 0.7214318513870239
Val Accuracy 0.5725677830940988
Best Val Accuracy 0.5725677830940988
Epoch 1
Train_accuracy 0.8863636363636364
Train Loss 0.30987101793289185
Val Accuracy 0.7033492822966507
Best Val Accuracy 0.7033492822966507
Epoch 2
Train_accuracy 0.9577352472089314
Train Loss 0.12823641300201416
Val Accuracy 0.8532695374800638
Best Val Accuracy 0.8532695374800638
Epoch 3
Train_accuracy 0.9772727272727273
Train Loss 0.06261049956083298
Val Accuracy 0.8373205741626795
Epoch 4
Train_accuracy 0.9856459330143541
Train Loss 0.03785247355699539
Val Accuracy 0.7751196172248804
Epoch 5
Train_accuracy 0.9924242424242424
Train Loss 0.03324383124709129
Val Accuracy 0.7990430622009569
Epoch 6
Train_accuracy 0.9904306220095693
Train Loss 0.027133379131555557
Val Accuracy 0.7655502392344498
Epoch 7
Train_accuracy 0.9952153110047847
Train Loss 0.017611326649785042
Val Accuracy 0.8229665071770335
Epoch 8
Train_accuracy 0.9860446570972887
Train 

In [ ]:
x = np.arange(1, len(hist)+1)

fig, ax = plt.subplots()  # Create a figure and an axes.
train_accuracy = list(map(lambda x: x['train_accuracy'], hist))
val_accuracy = list(map(lambda x: x['val_accuracy'], hist))

train_loss = list(map(lambda x: x['train_loss'], hist))
train_loss = train_loss / np.linalg.norm(train_loss)

plt.plot(x, train_accuracy, label='train accuracy')  # Plot some data on the axes.
plt.plot(x, val_accuracy, label='val accuracy')  # Plot more data on the axes...
plt.plot(x, train_loss, label='train loss')  # ... and some more.
plt.xlabel('#Epochs')  # Add an x-label to the axes.
plt.ylabel('Accuracy/Normalized Loss')  # Add a y-label to the axes.
plt.title("task2-resnet18-adam-30")  # Add a title to the axes.
plt.legend()  # Add a legend.

plt.annotate(f'{np.max(train_accuracy)}', xy=(np.argmax(train_accuracy) + 1, np.max(train_accuracy)), arrowprops=dict(facecolor='black', shrink=0.05))
plt.annotate(f'{np.max(val_accuracy)}', xy=(np.argmax(val_accuracy) + 1, np.max(val_accuracy)), arrowprops=dict(facecolor='black', shrink=0.05))
plt.show()